# Scrape Jobnet 

## Import packages

In [1]:
# import packages
import requests, json
import pandas as pd, numpy as np
import time, os


## Get web data

Procedure
collect the first page
save the totalresultcount for later usage


In [8]:
# the basic url
url = 'https://job.jobnet.dk/CV/FindWork/Search'


# the request session
session = requests.session()
session.headers['email'] = 'pot@potweb.dk' 
session.headers['name'] = 'Peter Ottosen'
#session.headers['User-Agent'] = '' # sometimes you need to pose as another agent...
session.headers

# container for jobpostings
jobpostings = []
#done = []

global last_t
last_t = time.time()

# function to create the next link
def create_link(offset_) :
    addit = '?Offset=' + str(offset) + '&SortValue=BestMatch'
    return url + addit

def ratelimit():
    time.sleep(2) # sleep one second.

# Reliable requests
def get(url,iterations=10,check_function=lambda x: x.ok):
    """This module ensures that your script does not crash from connection errors.
        that you limit the rate of your calls
        that you have some reliability check
        iterations : Define number of iterations before giving up. 
        exceptions: Define which exceptions you accept, default is all. 
    """
    for iteration in range(iterations):
        try:
            # add ratelimit function call here
            ratelimit() # !!
            response = session.get(url)
            if response.ok:
                return response # if succesful it will end the iterations here
        except exceptions as e: #  find exceptions in the request library requests.exceptions
            print(e) # print or log the exception message.
    return None # your code will purposely crash if you don't create a check function later.

def log_function(url,response,error_check=lambda x: x.ok,separator=',') :
    global last_t
    logfilepath = r"C:\Notebooks\group19\Eksamensprojekt/log_jobnet.csv"
    if os.path.isfile(logfilepath) :
        f_log = open(logfilepath,'a')
    else:
        f_log = open(logfilepath,'w') # define logfile, remember not to overwrite it.
        # write columns to be used, basic ones are, servertime, deltaT since last call, url, success of request, 
        header = ['serverTime','deltaT','url','success','length','path']
        f_log.write(','.join(header)+'\n')
    #### Update timing info ####
    t = time.time()
    delta_t = t-last_t # calculate time since last call
    last_t = t# update last call time
    #### meta data ### 
    success = error_check(response)
    if success: # if call is successfull we add it to the done container
        done.append(url)
    if response.ok:
        length = len(response.text)
    else:
        length = 0
    row = [t,delta_t,url,success,length,path]
    f_log.write(separator.join(map(str,row))+'\n')

# just to get the number of results
offset = 0
count = 0
r = get(create_link(offset))
d=json.loads(r.text)

ResultCount=d['TotalResultCount']
jp_keys = d['JobPositionPostings'][0].keys()

for pages in range(0, ResultCount, 20) : # 60 skal erstattes af ResultCount i en skarp version
    offset = pages
    link = create_link(offset)
    if link in done: #check if you have already downloaded the link
        continue
    count +=1
    r = get(link)
#    d=json.loads(r.text)
#    jobpostings.append(d['JobPositionPostings'])

    #define path 
    base_path = r"C:\Notebooks\group19\Eksamensprojekt/"
    path = base_path+str(count) # use the link count as a filename
    if r.ok:
        f = open(path,'w') 
        html = r.text
        try :
            f.write(html)
        except UnicodeEncodeError :
            print("Der opstod en UnicodeEncodeError ved skrivning af side", str(pages))
        else :
            pass
        f.write('\n\r')
        f.close()
    # run your log function
    log_function(link, r)

f_log.close()


Der opstod en UnicodeEncodeError ved skrivning af side 3760
Der opstod en UnicodeEncodeError ved skrivning af side 3780
Der opstod en UnicodeEncodeError ved skrivning af side 3860
Der opstod en UnicodeEncodeError ved skrivning af side 4080
Der opstod en UnicodeEncodeError ved skrivning af side 6340
Der opstod en UnicodeEncodeError ved skrivning af side 6560
Der opstod en UnicodeEncodeError ved skrivning af side 6840
Der opstod en UnicodeEncodeError ved skrivning af side 6940
Der opstod en UnicodeEncodeError ved skrivning af side 7060
Der opstod en UnicodeEncodeError ved skrivning af side 7780
Der opstod en UnicodeEncodeError ved skrivning af side 7840
Der opstod en UnicodeEncodeError ved skrivning af side 8260
Der opstod en UnicodeEncodeError ved skrivning af side 8340
Der opstod en UnicodeEncodeError ved skrivning af side 8760
Der opstod en UnicodeEncodeError ved skrivning af side 8820
Der opstod en UnicodeEncodeError ved skrivning af side 8880
Der opstod en UnicodeEncodeError ved skr

NameError: name 'f_log' is not defined

## Oversigt over data

In [95]:
print("Pages read:", len(jobpostings), type(jobpostings))
print("Postings per page:", len(jobpostings[1]), type(jobpostings[0]))
print("Number of variables in posting:", len(jobpostings[0][0]), type(jobpostings[0][0]))
for key in jp_keys :
    print(key, ":", jobpostings[0][0][key])

Pages read: 3 <class 'list'>
Postings per page: 20 <class 'list'>
Number of variables in posting: 42 <class 'dict'>
AutomatchType : 0
Weight : 1.0
Title : køkkenmedhjælper
JobHeadline : køkkenmedhjælper
Presentation : <p>Mama Rosa s&oslash;ger lige i &oslash;jeblikket 2 engagerede dygtige k&oslash;kkenmedhj&aelig;lper/pizzabagere med godt kendskab til det internationale k&oslash;kken.</p>
<p>Vi har fokus p&aring; k
HiringOrgName : MAMA ROSA COPENHAGEN APS
WorkPlaceAddress : Østergade 57
WorkPlacePostalCode : 1100
WorkPlaceCity : København K
WorkPlaceOtherAddress : False
WorkPlaceAbroad : False
WorkPlaceNotStatic : False
UseWorkPlaceAddressForJoblog : True
PostingCreated : 2018-08-23T00:00:00
LastDateApplication : 2018-10-18T00:00:00
AssignmentStartDate : 0001-01-01T00:00:00
IsHotjob : False
IsExternal : False
Url : https://job.jobnet.dk/CV/FindWork/JobDetail/4866851
Region : Hovedstaden og Bornholm
Municipality : København
Country : Danmark
PostalCode : 1100
PostalCodeName : None
JobAn

In [7]:
len(done)

156